In [2]:
import ltn
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Nodes information
raw_nodes = {
    "Hannover": (9.80, 52.39),
    "Frankfurt": (8.66, 50.14),
    "Hamburg": (10.08, 53.55),
    "Norden": (7.21, 53.60),
    "Bremen": (8.80, 53.08),
    "Berlin": (13.48, 52.52),
    "Muenchen": (11.55, 48.15),
    "Ulm": (9.99, 48.40),
    "Nuernberg": (11.08, 49.45),
    "Stuttgart": (9.12, 48.73),
    "Karlsruhe": (8.41, 49.01),
    "Mannheim": (8.49, 49.49),
    "Essen": (7.00, 51.44),
    "Dortmund": (7.48, 51.51),
    "Duesseldorf": (6.78, 51.22),
    "Koeln": (7.01, 50.92),
    "Leipzig": (12.38, 51.34)
}
# Convert raw_nodes to LTN constants
embedding_size = 2
nodes = {k: ltn.Constant(torch.rand((embedding_size,)), trainable=True) for k, v in raw_nodes.items()}
# nodes = {k: ltn.Constant(torch.tensor(v, device=device)) for k, v in raw_nodes.items()}


# Links information
raw_links = {
    "L1": ("Berlin", "Hamburg"),
    "L2": ("Berlin", "Hannover"),
    "L3": ("Berlin", "Leipzig"),
    "L4": ("Bremen", "Hamburg"),
    "L5": ("Bremen", "Hannover"),
    "L6": ("Bremen", "Norden"),
    "L7": ("Dortmund", "Essen"),
    "L8": ("Dortmund", "Hannover"),
    "L9": ("Dortmund", "Koeln"),
    "L10": ("Dortmund", "Norden"),
    "L11": ("Duesseldorf", "Essen"),
    "L12": ("Duesseldorf", "Koeln"),
    "L13": ("Frankfurt", "Hannover"),
    "L14": ("Frankfurt", "Koeln"),
    "L15": ("Frankfurt", "Leipzig"),
    "L16": ("Frankfurt", "Mannheim"),
    "L17": ("Frankfurt", "Nuernberg"),
    "L18": ("Hamburg", "Hannover"),
    "L19": ("Hannover", "Leipzig"),
    "L20": ("Karlsruhe", "Mannheim"),
    "L21": ("Karlsruhe", "Stuttgart"),
    "L22": ("Leipzig", "Nuernberg"),
    "L23": ("Muenchen", "Nuernberg"),
    "L24": ("Muenchen", "Ulm"),
    "L25": ("Nuernberg", "Stuttgart"),
    "L26": ("Stuttgart", "Ulm")
}
# define the link relation
links = list(raw_links.values())

In [4]:
def parse_demands(filepath):
    """
    Parse demands data from the text file and extract source, target, and value information
    
    Args:
        filepath (str): Path to the demand matrix file
        
    Returns:
        tuple: Three lists containing sources, targets, and values
    """
    sources = []
    targets = []
    values = []
    
    with open(filepath, 'r') as f:
        lines = f.readlines()
        demands_section = False
        
        for line in lines:
            # Start reading after DEMANDS section
            if line.strip() == "DEMANDS (":
                demands_section = True
                continue
            # Stop reading at the end of DEMANDS section    
            if line.strip() == ")":
                demands_section = False
                continue
                
            if demands_section and line.strip():
                # Parse each demand line
                # Example: Bremen_Nuernberg ( Bremen Nuernberg ) 1 0.011252 UNLIMITED
                parts = line.strip().split()
                if len(parts) >= 5:  # Ensure line has all required parts
                    source = parts[2]  # Bremen
                    target = parts[3]  # Nuernberg 
                    value = float(parts[6])  # 0.011252
                    
                    sources.append(source)
                    targets.append(target)
                    values.append(value)
    
    return sources, targets, values


filepath = "/home/zyang44/Github/baseline_cicIOT/Germany17/directed-nobel-germany-DFN-aggregated-5min-over-1day-native/demandMatrix-germany17-DFN-5min-20050215-0000.txt"
sources, targets, values = parse_demands(filepath)

# Combine into list of (source, target, value) tuples
demands = list(zip(sources, targets, values))

# Example to verify the data:
print(f"Total number of demands: {len(demands)}")
print("\nFirst 5 demands:")
for source, target, value in demands[:5]:
    print(f"{source} -> {target}: {value:.6f}")


Total number of demands: 251

First 5 demands:
Bremen -> Nuernberg: 0.011252
Bremen -> Hamburg: 0.090648
Berlin -> Dortmund: 0.525671
Stuttgart -> Nuernberg: 0.351524
Mannheim -> Duesseldorf: 0.000004


In [ ]:
# Create a link down event, for example, the link L1 from Berlin to Hamburg is down 
# 1. do a statistic on the demands, how many demands are affected by the link down event
# 2. Find affected demands where either source or target is Berlin or Hamburg
affected_demands = [d for d in demands if 
                   ('Berlin' in (d[0], d[1])) or ('Hamburg' in (d[0], d[1]))]

# Analyze the affected demands
print(f"Total affected demands: {len(affected_demands)}")

# Group and display demands by direction
print("\nDemands from/to Berlin:")
berlin_demands = [d for d in affected_demands if 'Berlin' in (d[0], d[1])]
for d in berlin_demands:
    print(f"{d[0]} -> {d[1]}: {d[2]:.6f}")

print("\nDemands from/to Hamburg:")
hamburg_demands = [d for d in affected_demands if 'Hamburg' in (d[0], d[1])]
for d in hamburg_demands:
    print(f"{d[0]} -> {d[1]}: {d[2]:.6f}")

# Direct demands between Berlin and Hamburg
direct_affected = [d for d in demands if 
                  (d[0] == 'Berlin' and d[1] == 'Hamburg') or 
                  (d[0] == 'Hamburg' and d[1] == 'Berlin')]
print("\nDirect demands between Berlin and Hamburg:")
for d in direct_affected:
    print(f"{d[0]} -> {d[1]}: {d[2]:.6f}")


Total affected demands: 62

Demands from/to Berlin:
Berlin -> Dortmund: 0.525671
Berlin -> Muenchen: 52.188255
Bremen -> Berlin: 0.150218
Berlin -> Bremen: 0.651607
Nuernberg -> Berlin: 1.386174
Hannover -> Berlin: 8.071638
Berlin -> Hannover: 10.077060
Muenchen -> Berlin: 5.956407
Berlin -> Koeln: 1.633112
Berlin -> Duesseldorf: 0.000680
Berlin -> Essen: 0.086774
Leipzig -> Berlin: 10.534064
Berlin -> Frankfurt: 174.138905
Berlin -> Hamburg: 2.838691
Berlin -> Norden: 0.000122
Dortmund -> Berlin: 0.601435
Hamburg -> Berlin: 0.384028
Karlsruhe -> Berlin: 0.014829
Koeln -> Berlin: 0.956643
Berlin -> Stuttgart: 0.124679
Frankfurt -> Berlin: 69.843559
Mannheim -> Berlin: 0.031183
Berlin -> Leipzig: 46.194475
Berlin -> Mannheim: 0.088746
Berlin -> Nuernberg: 0.390074
Duesseldorf -> Berlin: 0.006202
Norden -> Berlin: 0.000169
Berlin -> Ulm: 0.000508
Berlin -> Karlsruhe: 0.565055
Ulm -> Berlin: 0.001881
Essen -> Berlin: 0.066039
Stuttgart -> Berlin: 41.384327

Demands from/to Hamburg:
Bremen

In [ ]:
# the link down event only effects the links that are directly connected to the link that is down
# 1. Find the links that are directly connected to the link that is down
# 2. Find the demands that are affected by the link down event

# Find the links that are directly connected to the link that is down
down_link = ("Berlin", "Hamburg")
affected_links = [l for l in links if l[0] in down_link or l[1] in down_link]
print("\nLinks directly connected to the down link:")
for l in affected_links:
    print(f"{l[0]} -> {l[1]}")

# So there are 4 links directly connected to the down link, which are: L1, L2, L18, L4 (assign to the affected_links)
# 3 nodes are directly connected to the down link, which are: Berlin, Hamburg, Hannover, Bremen, and Leipzig (assign to the affected_nodes)


Links directly connected to the down link:
Berlin -> Hamburg
Berlin -> Hannover
Berlin -> Leipzig
Bremen -> Hamburg
Hamburg -> Hannover


In [ ]:
# Assume the node Berlin is down, we need to find the affected demands
# 1. Find the affected links
# 2. Find the affected demands
down_node = 'Frankfurt'
down_links = [l for l in links if down_node in l]
print("Down links:")
for l in down_links:
    print(f"{l[0]} -> {l[1]}")

Down links:
Frankfurt -> Hannover
Frankfurt -> Koeln
Frankfurt -> Leipzig
Frankfurt -> Mannheim
Frankfurt -> Nuernberg
